In [1]:
# Imports
import duckdb
import pandas as pd

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

kb_querys = [
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_chaves_na_mao.chaves_na_mao_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_viva_real.viva_real_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_zap_imoveis.zap_imoveis_register"
]

# Fetch knowledge base data
df_kb_list = [con.execute(query).fetch_df() for query in kb_querys]

# Concatenate knowledge base data
df_kb = pd.concat(df_kb_list, ignore_index=True)

In [6]:
df_kb["bairro"].value_counts(normalize=True, ascending=True)

bairro
lamenha pequena      0.000214
sao miguel           0.000643
augusta              0.000857
cascatinha           0.001929
orleans              0.002358
                       ...   
xaxim                0.040943
agua verde           0.046088
uberaba              0.049732
cidade industrial    0.049946
centro               0.055734
Name: proportion, Length: 67, dtype: float64

In [8]:
import duckdb

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Definir a quaery
query = """
DROP SCHEMA kodomiya_leilao_imovel CASCADE;
"""

#con.execute(query)

In [11]:
import duckdb
from datetime import datetime

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Executar uma query no banco
today = datetime.now().strftime("%Y-%m-%d")
df_leilao = con.execute(f"SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register WHERE data_primeira_praca	< '{today}' OR data_segunda_praca < '{today}'").fetchdf()

In [9]:
df_leilao[["bairro", "rua", "cidade", "latitude", "longitude"]]

,bairro,rua,cidade,latitude,longitude
0,umbara,RUA DILSON LUIZ,Curitiba,NaN,NaN
1,uberaba,RUA RODOLFO BERNARDELLI,Curitiba,-25.489124,-49.226351


In [12]:
df_leilao["bairro"].value_counts()

bairro
umbara     1
uberaba    1
Name: count, dtype: int64

In [5]:
import duckdb
from datetime import datetime

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Executar uma query no banco
today = datetime.now().strftime("%Y-%m-%d")
df_leilao = con.execute(f"SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register").fetchdf()